In [1]:
from correctingagent.world.world import *
from correctingagent.agents.teacher import *
from correctingagent.world.rules import Rule
import os
import correctingagent.pddl.ff as ff

from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import GibbsSampling
from pgmpy.models import FactorGraph

from correctingagent.models.pgmmodels import *

from correctingagent.models.pgmmodels import CorrectionPGMModel, PGMPYInference, InferenceType, get_scope, is_overlap, combine_models, SamplingType, to_CPD, reduce_model, get_non_zero_states
from correctingagent.models.prob_model import KDEColourModel
from correctingagent.world import rules
from correctingagent.world.rules import RedOnBlueRule, CorrectionType
from correctingagent.util.CPD_generation import variable_or_CPD, generate_neg_table_cpd, equals_CPD


from collections import namedtuple
import itertools

import networkx as nx
import numpy as np

from pgmpy.factors import factor_product
from pgmpy.inference import Inference
from pgmpy.models import BayesianModel, MarkovChain, MarkovModel
from pgmpy.utils.mathext import sample_discrete
from pgmpy.extern.six.moves import map, range
from pgmpy.sampling import _return_samples

from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import State
from pgmpy.factors.continuous import ContinuousFactor
from functools import reduce
from operator import mul
from queue import PriorityQueue
import heapq

In [4]:
pgm_model = CorrectionPGMModel(inference_type=InferenceType.SearchInference,
                      max_inference_size=1, max_beam_size=2)

red_cm = KDEColourModel('red')
blue_cm = KDEColourModel('blue')

time = 0
red_on_blue_rules = rules.Rule.generate_red_on_blue_options('red', 'blue')

violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b1', 'b2'], time,
                                     correction_type=CorrectionType.TOWER)

pgm_model.observe({'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], f'corr_{time}': 1})


q = pgm_model.query(violations)
# inference = PGMPYInference(pgm_model)
# inference.infer({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
# q = inference.query(violations)
# #
# q = pgm_model.query(violations, [1, 1])
#
# assert(q[violations[0]] - 0.5 < 0.2)
# assert(q[violations[1]] - 0.5 < 0.2)
#
#


time = 1
violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b3', 'b4'], time,
                                    correction_type=CorrectionType.TOWER)

pgm_model.observe({'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], f'corr_{time}': 1})

q = pgm_model.query(violations)

assert (q[violations[0]] - 0.5 < 0.2)
assert (q[violations[1]] - 0.5 < 0.2)

print(q)

pgm_model.observe({'red(b3)': 1})

q = pgm_model.query(violations)

assert (q[violations[0]] == 1.0)
assert (q[violations[1]] == 0.0)

fix bw? False
bw 0.5
fix bw? False
bw 0.5
{'V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0.5, 'V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 0.5}


In [3]:
evidence = {'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], f'corr_0': 1, 'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], f'corr_1': 1, "blue(b2)":1}
model, additional_vars = reduce_model(pgm_model.ordered_models[0], evidence=evidence)

In [70]:
beams = get_non_zero_states(pgm_model.ordered_models[0], evidence)
beams = update_beams([(1, SortableDict())], beams, evidence, pgm_model.ordered_models[:1])
beams = normalise_beam(beams)
print(beams)
new_beams = get_non_zero_states(pgm_model.ordered_models[1], evidence)
beams = update_beams(beams, new_beams, evidence, pgm_model.ordered_models, max_size=2)
beams = normalise_beam(beams)
print(beams)

[(0.10000000000000002, {'red(b1)': 0, 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 1, 'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1, 'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], 'corr_0': 1, 'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], 'corr_1': 1, 'blue(b2)': 1}), (0.9, {'red(b1)': 0, 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 0, 'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1, 'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], 'corr_0': 1, 'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], 'corr_1': 1, 'blue(b2)': 1})]
[(0.10000000000000002, {'red(b1)': 0, 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 1, 'all y.(blue(y) -> exists x. (red

In [71]:
for v, b in beams:
    print(v, b)

0.10000000000000002 {'red(b1)': 0, 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 1, 'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1, 'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], 'corr_0': 1, 'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], 'corr_1': 1, 'blue(b2)': 1, 'V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 0, 'V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 1, 'red(b3)': 1, 'blue(b4)': 0}
0.9 {'red(b1)': 0, 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 0, 'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1, 'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], 'corr_0': 1, 'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], 'corr_1': 1, 'blue(b2)': 1, 'V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1, 'V_1(all x.(red(x) -> exists y. (blue(y) & on(

In [40]:
sum([v for v, b in beams])

1.0

In [30]:
normalise_beam(beams)

[({'red(b1)': 0,
   'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1,
   'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0,
   'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 0,
   'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1,
   'F(b3)': [1, 1, 1],
   'F(b4)': [0, 0, 0],
   'corr_0': 1,
   'F(b1)': [1, 1, 1],
   'F(b2)': [0, 0, 0],
   'corr_1': 1,
   'blue(b2)': 1},
  0.9),
 ({'red(b1)': 0,
   'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 1,
   'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0,
   'all x.(red(x) -> exists y. (blue(y) & on(x,y)))': 1,
   'all y.(blue(y) -> exists x. (red(x) & on(x,y)))': 1,
   'F(b3)': [1, 1, 1],
   'F(b4)': [0, 0, 0],
   'corr_0': 1,
   'F(b1)': [1, 1, 1],
   'F(b2)': [0, 0, 0],
   'corr_1': 1,
   'blue(b2)': 1},
  0.10000000000000002)]

In [8]:
b = beams[0][0]
evaluate_model(b, model)

IndexError: invalid index to scalar variable.

In [9]:
m = build_combined_model(pgm_model.ordered_models)

In [10]:
get_scope(m)

{'F(b1)',
 'F(b2)',
 'F(b3)',
 'F(b4)',
 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))',
 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))',
 'V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y))))',
 'V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y))))',
 'all x.(red(x) -> exists y. (blue(y) & on(x,y)))',
 'all y.(blue(y) -> exists x. (red(x) & on(x,y)))',
 'blue(b2)',
 'blue(b4)',
 'corr_0',
 'corr_1',
 'red(b1)',
 'red(b3)'}

In [11]:
m.factors

[<DiscreteFactor representing phi(V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b3):2, blue(b4):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f15ee17cbd0>,
 <DiscreteFactor representing phi(V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2, red(b3):2, blue(b4):2, all y.(blue(y) -> exists x. (red(x) & on(x,y))):2) at 0x7f15ee17c210>,
 <DiscreteFactor representing phi(corr_0:2, V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2) at 0x7f15ee17cc10>,
 <DiscreteFactor representing phi(V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b1):2, blue(b2):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f15ee17ca90>,
 <DiscreteFactor representing phi(all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f15ee17c750>,
 <DiscreteFactor representing phi(corr_1:2, V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2) at 0x7f15ee

In [74]:
b.update(evidence)
evaluate_factor(b, m.factors[5])

0.5

In [9]:
q = PriorityQueue()


In [ ]:
q.put